In [16]:
import open3d as o3d
import numpy as np
def get_tree_from_coord(pcd, grd_pcd, coord:list, expand_x_y:list=[10.0,10.0], expand_z:list=[-10.0,10.0]):
    # CAREFUL THE Y IS ACTUALLY NEGATIVE
    xc, yc = coord[0], -coord[1]
    
    l,w = expand_x_y
    zmin, zmax = expand_z
    bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(xc-l/2,yc-w/2,zmin),max_bound=(xc+l/2,yc+w/2,zmax))
    ground = grd_pcd.crop(bbox)
    
    # Removing Outlier by taking the minimum z value of Ground from CSF FILTER
    zmin = ground.get_min_bound()[2]
    zmin_tolerance = ground.get_max_bound()[2] - 2.0
    zmin = zmin_tolerance if zmin > zmin_tolerance else zmin
    bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(xc-l/2,yc-w/2,zmin),max_bound=(xc+l/2,yc+w/2,zmax))
    tree = pcd.crop(bbox) + ground
    return tree

In [17]:
pcd = o3d.io.read_point_cloud("model_dense4k10fps1Tree.ply", format='auto')

In [ ]:
center = pcd.get_center()[0:2]
center[1]*=-1
side_view_img_size = [320, 768]
side_view_step_size = 0.02
ex_w, ex_h = (dim*side_view_step_size for dim in side_view_img_size)

almost_tree = get_tree_from_coord(pcd, pcd, center, expand_x_y=[ex_w,ex_w], 
                                                    expand_z=[-100, 100]
                                                    )

In [ ]:
non_grd = pcd
# Expand search radius of new center by 2 meters with how many linspace?
expand_meters = 2
x_min_pcd, y_min_pcd, z_min = non_grd.get_min_bound()
x_max_pcd, y_max_pcd, z_max = non_grd.get_max_bound()

new_min, new_max = center-expand_meters/2 , center+expand_meters/2

# Find new x_coords
new_x_list, x_increment= np.linspace(new_min[0], new_max[0], 8, retstep=True)
new_y_list, y_increment = np.linspace(new_min[1], new_max[1], 8, retstep=True)

for i, new_x in enumerate(x_arr_pcd):
    for i, new_y in enumerate(y_arr_pcd):
        new_center = (center[0],-y)
        new_expand = (5, y_increment*2)
        single_slice = get_tree_from_coord(
            almost_tree, 
            almost_tree, 
            new_center, 
            expand_x_y=new_expand,
            expand_z=[-100, 100]
            )
        o3d.visualization.draw_geometries([single_slice])

[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.


In [ ]:
o3d.visualization.draw_geometries([almost_tree])

In [42]:
print(ex_w,ex_h)

6.4 15.36
